# 📘 代理架构 8：情景+语义记忆栈

欢迎来到我们系列的第八个notebook。今天，我们将处理创建真正智能的长期助手时最关键的挑战之一：**持久记忆**。标准聊天机器人记忆是短暂的，只持续一个会话。要构建一个与用户一起学习和成长的个性化代理，我们需要一个更健壮的解决方案。

我们将实现一个镜像人类认知的结构化记忆架构，结合两种不同类型的记忆：

1. **情景记忆：** 这是特定事件或过去交互的记忆。它回答问题"发生了什么？"（例如，"上周，用户问我关于NVIDIA的股价。"）。我们将使用**向量数据库**来查找与当前主题相关的过去对话。
2. **语义记忆：** 这是从那些事件中提取的结构化事实、概念和关系的记忆。它回答问题"我知道什么？"（例如，"用户Alex是一个保守型投资者。"，"Alex对科技股感兴趣。"）。我们将使用**图数据库（Neo4j）**，因为它擅长管理和查询复杂关系。

通过结合这些，我们的代理不仅可以回忆过去的对话，还可以构建关于用户和世界的丰富、互联的知识库，从而实现深度个性化和上下文感知的交互。

### 定义
**情景+语义记忆栈**是一种代理架构，维护两种类型的长期记忆。**情景记忆**存储经历的时间顺序日志（例如，聊天历史摘要），通常基于语义相似性进行搜索。**语义记忆**在知识库中存储提取的结构化知识（事实、实体、关系），通常是图的形式。

### 高级工作流程

1. **交互：** 代理与用户进行对话。
2. **记忆检索（回忆）：** 对于新的用户查询，代理首先查询两个记忆系统。
   * 它搜索**情景**向量存储以查找类似的过去对话。
   * 它查询**语义**图数据库以查找与查询相关的实体和事实。
3. **增强生成：** 检索到的记忆被添加到提示的上下文中，允许LLM生成一个了解过去交互和学习到的事实的响应。
4. **记忆创建（编码）：** 交互完成后，后台进程分析对话。
   * 它创建该轮对话的简洁摘要（新的**情景**记忆）。
   * 它提取关键实体和关系（新的**语义**记忆）。
5. **记忆存储：** 新的情景摘要被嵌入并保存到向量存储。新的语义事实作为节点和边写入图数据库。

### 何时使用/应用场景
* **长期个人助手：** 一个能记住你的偏好、项目和个人详情数周或数月的助手。
* **个性化系统：** 记住你风格的电商机器人，或记住你学习进度和薄弱点的教育导师。
* **复杂研究代理：** 一个在探索文档时构建主题知识图的代理，允许它回答复杂的多跳问题。

### 优点和缺点
* **优点：**
  * **真正的个性化：** 实现无限期持续的上下文和学习，远超单个会话的上下文窗口。
  * **丰富的理解：** 图数据库允许代理理解和推理实体之间的复杂关系。
* **缺点：**
  * **复杂性：** 这是一个比简单无状态代理更复杂的架构，需要构建和维护。
  * **记忆膨胀和修剪：** 随着时间推移，记忆存储可能变得庞大。总结、合并或修剪旧的/不相关记忆的策略对于长期性能至关重要。

## 阶段0：基础与设置

我们将安装所有必要的库，包括向量和图数据库的驱动程序，并配置我们的API密钥。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain_community langchain-openai neo4j faiss-cpu tiktoken

In [2]:
import os
import uuid from typing import List, Dict, Any, Optional, Tuple from dotenv import load_dotenv

# Pydanticfordata modeling from pydantic import BaseModel, Field

# LangChain components from langchain_nebius import ChatNebius, NebiusEmbeddings from langchain_community.graphs import Neo4jGraph from langchain_community.vectorstores import FAISS from langchain.docstore.document import Document from langchain_core.prompts import ChatPromptTemplate

# LangGraph components from langgraph import StateGraph, END from typing_extensions import TypedDict

# 用于美观打印 from rich.console import Console from rich.markdown import Markdown

# --- API KeyandTracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Memory Stack (Nebius)"

# Checkforrequired environment variables
required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "NEO4J_URI", "NEO4J_USERNAME", "NEO4J_PASSWORD"] for var in required_vars:
 if var not in os.environ:
 print(f"警告: Environment variable {var} not set.")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1：构建记忆组件

这是我们架构的核心。我们将定义记忆的结构并设置与数据库的连接。我们还将创建负责处理对话和创建新记忆的"记忆制造者"代理。

In [ ]:
console = Console()
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
embeddings = NebiusEmbeddings()

# --- 1. Vector StoreforEpisodic Memory ---
# 在实际应用中，你会持久化这个。对于这个示例，它是内存中的。
try:
 episodic_vector_store = FAISS.from_texts(["初始文档用于引导存储"], embeddings)
except ImportError:
 console.print("[bold red]FAISS未安装。请运行`pip install faiss-cpu`。")
 episodic_vector_store = None

# --- 2. Graph DBforSemantic Memory ---
try:
 graph = Neo4jGraph(
 url=os.environ.get("NEO4J_URI"),
 username=os.environ.get("NEO4J_USERNAME"),
 password=os.environ.get("NEO4J_PASSWORD")
 )
 # 清除Graph以进行干净的运行
.query("MATCH (n) DETACH DELETE n")
except Exception as e:
 console.print(f"[bold red]连接Neo4j失败：{e}。请检查您的凭据和连接。[/bold red]")
 graph = None

# --- 3. Pydantic Modelsfor"Memory Maker" ---
# 定义我们想要提取的知识结构。
class Node(BaseModel):
 id: str = Field(description="Node的唯一标识符，可以是人名、公司代码或概念。")
 type: str = Field(description="Node类型（例如，'User'、'Company'、'InvestmentPhilosophy'）。")
 properties: Dict[str, Any] = Field(description="Node属性的字典。")

class Relationship(BaseModel):
 source: Node = Field(description="关系的源Node。")
 target: Node = Field(description="关系的目标Node。")
 type: str = Field(description="关系类型（例如，'IS_A'、'INTERESTED_IN'）。")
 properties: Dict[str, Any] = Field(description="关系属性的字典。")

class KnowledgeGraph(BaseModel):
 """表示从对话中提取的结构化知识。"""
 relationships: List[Relationship] = Field(description="要添加到知识Graph的关系列表。")

# --- 4. "Memory Maker" Agent ---
def create_memories(user_input: str, assistant_output: str):
 conversation = f"User: {user_input}\nAssistant: {assistant_output}"
 
 # 4a. 创建情景记忆（摘要）
 console.print("--- 创建情景记忆（摘要） ---")
 summary_prompt = ChatPromptTemplate.from_messages([
 ("system", "你是一个摘要专家。为以下用户-助手交互创建一个简洁的单句摘要。这个摘要将用作未来回忆的记忆。"),
 ("human", "Interaction:\n{interaction}")
 ])
 summarizer = summary_prompt | llm
 episodic_summary = summarizer.invoke({"interaction": conversation}).content
 
 new_doc = Document(page_content=episodic_summary, metadata={"created_at": uuid.uuid4().hex})
 episodic_vector_store.add_documents([new_doc])
 console.print(f"[green]Episodic memory created:[/green] '{episodic_summary}'")
 
 # 4b. 创建语义记忆（事实提取）
 console.print("--- 创建语义记忆（Graph） ---")
 extraction_llm = llm.with_structured_output(KnowledgeGraph)
 extraction_prompt = ChatPromptTemplate.from_messages([
 ("system", "你是一个知识提取专家。你的任务是从对话中识别关键实体及其关系，并将它们建模为Graph。专注于用户偏好、目标和陈述的事实。"),
 ("human", "从这个交互中提取所有关系：\n{interaction}")
 ])
 extractor = extraction_prompt | extraction_llm
 try:
 kg_data = extractor.invoke({"interaction": conversation})
 if kg_data.relationships:
 for rel in kg_data.relationships:
.add_graph_documents([rel], include_source=True)
 console.print(f"[green]Semantic memory created:[/green] Added {len(kg_data.relationships)} 个关系到Graph。")
 else:
 console.print("[yellow]在这个交互中没有识别到新的语义记忆。[/yellow]")
 except Exception as e:
 console.print(f"[red]无法提取或保存语义记忆： {e}[/red]")

if episodic_vector_storeandgraph:
 print("记忆组件初始化成功。")

记忆组件初始化成功。


## 阶段2：记忆增强代理

现在我们将构建使用这个记忆系统的代理。我们将使用LangGraph定义一个清晰的、有状态的工作流程：检索记忆，使用这些记忆生成响应，最后用最新的交互更新记忆。

In [4]:
# 为我们的LangGraph代理定义状态
class AgentState(TypedDict):
 user_input: str
 retrieved_memories: Optional[str]
 generation: str

# 定义Graph的Node

def retrieve_memory(state: AgentState) -> Dict[str, Any]:
 """Node 那个retrieves memoriesfromboth episodicandsemantic stores."""
 console.print("--- 检索记忆 ---")
 user_input = state['user_input']
 
 # 从情景记忆中检索
 retrieved_docs = episodic_vector_store.similarity_search(user_input, k=2)
 episodic_memories = "\n".join([doc.page_contentfor doc in retrieved_docs])
 
 # 从语义记忆中检索
 # 这是一个简单的检索；更高级的方法会涉及从查询中提取实体
 try:
 graph_schema =.get_schema
 # 使用全文索引以获得更好的检索。Neo4j会自动在Node属性中创建一个。
 # 更健壮的解决方案可能涉及首先从user_input中提取实体。
 semantic_memories = str(.query("""
 UNWIND $keywords AS keyword
 CALL db.index.fulltext.queryNodes("entity", keyword) YIELD node, score
 MATCH (node)-[r]-(related_node)
 RETURN node, r, related_node LIMIT 5
 """, {'keywords': user_input.split()}))
 except Exception as e:
 semantic_memories = f"无法查询Graph：{e}"
 
 retrieved_content = f"相关的过去对话（情景记忆）：\n{episodic_memories}\n\nRelevant Facts (Semantic Memory):\n{semantic_memories}"
 console.print(f"[cyan]检索到的上下文：\n{retrieved_content}[/cyan]")
 
 return{"retrieved_memories": retrieved_content}

def generate_response(state: AgentState) -> Dict[str, Any]:
 """Node 那个generates 一个response using retrieved memories."""
 console.print("--- 生成响应 ---")
 prompt = ChatPromptTemplate.from_messages([
 ("system", "你是一个有帮助的个性化金融助手。使用检索到的记忆来指导你的响应并为用户量身定制。如果记忆表明用户的偏好（例如，他们是保守型投资者），你必须尊重它。"),
 ("human", "我的问题是：{user_input}\n\n以下是一些可能相关的记忆：\n{retrieved_memories}")
 ])
 generator = prompt | llm
 generation = generator.invoke(state).content
 console.print(f"[green]生成的响应：\n{generation}[/green]")
 return{"generation": generation}

def update_memory(state: AgentState) -> Dict[str, Any]:
 """Node 那个updates memorywithlatest interaction."""
 console.print("--- 更新记忆 ---")
 create_memories(state['user_input'], state['generation'])
 return{}

# 构建Graph
workflow = StateGraph(AgentState)

workflow.add_node("retrieve", retrieve_memory)
workflow.add_node("generate", generate_response)
workflow.add_node("update", update_memory)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", "update")
workflow.add_edge("update", END)

memory_agent = workflow.compile()
print("记忆增强代理Graph编译成功。")

记忆增强代理图编译成功。


## 阶段3：演示与检查

让我们看看代理的实际运行。我们将模拟一个多轮对话。前两轮将播种记忆。第三轮将测试代理是否能使用该记忆进行个性化响应。最后，我们将直接检查数据库以查看创建的记忆。

In [5]:
def run_interaction(query: str):
 result= memory_agent.invoke({"user_input": query})
 return result['generation']

console.print("\n--- 💬 INTERACTION 1: Seeding Memory ---")
run_interaction("Hi, my name is Alex. I'm 一个conservative investor,andI'm mainly interestedinestablished tech companies.")

console.print("\n--- 💬 INTERACTION 2: Asking 特定的问题---")
run_interaction("What do you think 关于Apple (AAPL)?")

console.print("\n--- 🧠 INTERACTION 3: THE MEMORY TEST ---")
run_interaction("基于我的目标, what's 一个good alternative来那个stock?")


--- 💬 交互1：播种记忆 ---


--- 检索记忆 ---
检索到的上下文：
相关的过去对话（情景记忆）：
初始文档用于引导存储

相关事实（语义记忆）：
[]
--- 生成响应 ---
生成的响应：
Hello, Alex! It's great来meet you. As 一个conservative investor, focusinginestablished tech companieswithstrong fundamentals is 一个very sound 策略. I 可以certainly help you navigate 那个space. What'sinyour mind today?
--- 更新记忆 ---
--- 创建情景记忆（摘要） ---
情景记忆已创建： '用户Alex介绍自己是一个对成熟科技公司感兴趣的保守型投资者。'
--- 创建语义记忆（图） ---
语义记忆已创建： 添加了 2 个关系到图。



--- 💬 交互2：询问具体问题 ---


--- 检索记忆 ---
检索到的上下文：
相关的过去对话（情景记忆）：
用户Alex介绍自己是一个对成熟科技公司感兴趣的保守型投资者。
初始文档用于引导存储

相关事实（语义记忆）：
[]
--- 生成响应 ---
生成的响应：
Apple (AAPL) is often considered 一个cornerstoneforconservative tech portfolios. It has 一个massive market capitalization, 一个very strong brand, consistent profitability,and一个history的returning value来shareholders through dividendsandbuybacks. Its ecosystem的products creates 一个loyal customer base, which 提供 一个stable revenue stream. for一个conservative investor, it generally aligns wellwithgoal的capital preservation while still offering growth potential. Would you like 一个deeper dive into its recent performanceorfinancials?
--- 更新记忆 ---
--- 创建情景记忆（摘要） ---
情景记忆已创建： '用户询问了Apple (AAPL)，助手确认它是适合保守型投资者的股票，因为其稳定性和市场地位。'
--- 创建语义记忆（图） ---
语义记忆已创建： 添加了 1 个关系到图。



--- 🧠 交互3：记忆测试 ---


--- 检索记忆 ---
检索到的上下文：
相关的过去对话（情景记忆）：
用户Alex介绍自己是一个对成熟科技公司感兴趣的保守型投资者。
用户询问了Apple (AAPL)，助手确认它是适合保守型投资者的股票，因为其稳定性和市场地位。

相关事实（语义记忆）：
[{'node': {'type': 'User', 'id': 'Alex', 'properties': {}}, 'r': {'type': 'HAS_GOAL', 'properties': {}}, 'related_node': {'type': 'InvestmentPhilosophy', 'id': 'Conservative Investing', 'properties': {}}}, {'node': {'type': 'User', 'id': 'Alex', 'properties': {}}, 'r': {'type': 'INTERESTED_IN', 'properties': {}}, 'related_node': {'type': 'Sector', 'id': 'Tech', 'properties': {}}}]
--- 生成响应 ---
生成的响应： 的course. Basedinyour stated goal的conservative investingintech sector, 一个great alternative来Apple (AAPL) would be Microsoft (MSFT).

Here's why it fits your profile:
1. **Diversification:** Like Apple, it's 一个mega-cap tech giant，但its revenue is more diversified across enterprise software (Azure, Office 365), gaming (Xbox),andhardware (Surface).
2. **Strong Enterprise Focus:** Its dominanceincloud computingwithAzure 提供 一个consistentandgrowing revenue stream, which 

### 检查记忆存储

让我们看看底层。我们可以直接查询数据库来查看代理创建的记忆。

In [6]:
console.print("--- 🔍 Inspecting Episodic Memory (Vector Store) ---")
# 我们将执行一个相似性搜索，以查看一般概念的结果
retrieved_docs = episodic_vector_store.similarity_search("User's investment 策略", k=3) fori, docinenumerate(retrieved_docs):
 print(f"{i+1}. {doc.page_content}")

console.print("\n--- 🕸️ Inspecting Semantic Memory (Graph database) ---")
print(f"Graph Schema:\n{.get_schema}")

# Cypher 查询以查看谁对什么感兴趣
query_result =.query("MATCH (n:User)-[r:INTERESTED_IN|HAS_GOAL]->(m) RETURN n, r, m")
print(f"RelationshipsinGraph:\n{query_result}")

--- 🔍 检查情景记忆（向量存储） ---


1. 用户Alex介绍自己是一个对成熟科技公司感兴趣的保守型投资者。
2. 基于用户的保守投资目标，助手建议微软(MSFT)作为Apple (AAPL)的良好替代品，强调其多元化和企业重点。
3.用户inquired 关于Apple (AAPL),andassistant confirmed it's 一个suitable stockforconservative investors due来its stabilityand市场地位.



--- 🕸️ 检查语义记忆（图数据库） ---


图模式：
{'node_props': {'InvestmentPhilosophy': [{'property': 'id', 'type': 'STRING'}], 'Company': [{'property': 'id', 'type': 'STRING'}], 'Sector': [{'property': 'id', 'type': 'STRING'}], 'User': [{'property': 'id', 'type': 'STRING'}]}, 'rel_props': {}, 'relationships': [{'start': 'User', 'type': 'INTERESTED_IN', 'end': 'Sector'}, {'start': 'User', 'type': 'INTERESTED_IN', 'end': 'Company'}, {'start': 'User', 'type': 'HAS_GOAL', 'end': 'InvestmentPhilosophy'}]}

图中的关系：
[{'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'Conservative Investing', 'type': 'InvestmentPhilosophy'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'Tech', 'type': 'Sector'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'AAPL', 'type': 'Company'}}, {'n': {'id': 'Alex', 'type': 'User'}, 'r': {}, 'm': {'id': 'MSFT', 'type': 'Company'}}]


## 总结

在这个notebook中，我们成功构建了一个具有复杂长期记忆系统的代理。演示清楚地展示了这种架构的强大：

- **无状态失败：** 一个标准代理，当被问到"基于我的目标，有什么好的替代品？"时，会失败，因为它没有用户目标的记忆。
- **记忆增强成功：** 我们的代理成功了，因为它可以：
  1. **情景回忆：** 它检索了第一次对话的摘要："用户Alex介绍自己是一个保守型投资者..."
  2. **语义回忆：** 它查询图并找到结构化事实：`(User: Alex) -[HAS_GOAL]-> (InvestmentPhilosophy: Conservative)`。
  3. **综合：** 它使用这个组合上下文提供了一个高度相关和个性化的推荐（微软），明确引用了用户的保守目标。

这种结合回忆*发生了什么*（情景）和*知道什么*（语义）的方式是一个强大的范式，用于超越简单的事务性代理，创建真正的学习伙伴。虽然大规模管理这种记忆存在修剪和合并等挑战，但我们在这里构建的基础架构是迈向更智能和个性化AI系统的重要一步。